The low correlation between variable lead us to try to build five different models. Three of them would predict
a single variable using only its past values. Two of them would predict a couple of highly correlated variables
using only past values of that specific couple of variable (i.e. Hype Root and Crunchiness to predict Hype root and
Crunchiness)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset = pd.read_csv('../input/training/Training.csv')
print(dataset.shape)
dataset.head()

In [ ]:
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
inspect_dataframe(dataset, dataset.columns)

In [ ]:
sponginess=tf.expand_dims(dataset["Sponginess"],axis=1)
crunch_hype=tf.concat([tf.expand_dims(dataset["Crunchiness"],axis=1),tf.expand_dims(dataset["Hype root"],axis=1)],axis=1)
wonder_loud=tf.concat([tf.expand_dims(dataset["Wonder level"],axis=1),tf.expand_dims(dataset["Loudness on impact"],axis=1)],axis=1)                       
meme_creativity=tf.expand_dims(dataset["Meme creativity"],axis=1)
soap_slipperiness=tf.expand_dims(dataset["Soap slipperiness"],axis=1)
#Wonder level	Crunchiness	Loudness on impact	Meme creativity	Soap slipperiness	Hype root
print(sponginess.shape)
single_datasets=[sponginess,meme_creativity,soap_slipperiness]
couple_datasets=[crunch_hype,wonder_loud]
single_names=["sponginess","meme_creativity","soap_slipperiness"]
couple_names=["crunch_hype","wonder_loud"]


In [ ]:
window = 300
stride = 20

In [ ]:
future = dataset[-window:]
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
def build_sequences(df, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df
    temp_label = df
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
target_labels = dataset.columns
telescope = 100

In [ ]:
train_sequences=[]
names=[]

i=0
for data in single_datasets:
    X_train, y_train = build_sequences(data, window, stride, telescope)
    print(X_train.shape, y_train.shape)
    train_sequences.append([X_train,y_train])
    names.append(single_names[i])
    i=i+1
i=0
for data in couple_datasets:
    X_train, y_train = build_sequences(data, window, stride, telescope)
    print(X_train.shape, y_train.shape)
    train_sequences.append([X_train,y_train])
    names.append(couple_names[i])
    i=i+1


In [ ]:
def inspect_multivariate(X, y, columns, telescope, name,idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))
        col=[]
    for i in range(columns):
        col.append("bruh"+str(i))
    if len(col)>1:
        figs, axs = plt.subplots(len(col), 1, sharex=True, figsize=(17,5))
        for i, col1 in enumerate(col):
            print(i)
            print(col1)
            axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
            axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
            axs[i].set_title(name)
            #axs[i].set_ylim(0,1)
    else:
        plt.figure(figsize=(17,2.5))
        plt.plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        plt.scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        plt.title(name)
    
    plt.show()

In [ ]:
i=0
print(len(train_sequences))
for seq in train_sequences:
    name=names[i]
    print(name)
    inspect_multivariate(seq[0], seq[1],seq[0].shape[2], telescope, name=name)
    i=i+1

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape,learning_rate):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(256, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5)(convlstm)

    # In order to predict the next values for more than one channel,
    # we can use a Dense layer with a number given by telescope*num_channels,
    # followed by a Reshape layer to obtain a tensor of dimension 
    # [None, telescope, num_channels]
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics=['mae'])

    # Return the model
    return model

In [ ]:
single_input_shape = (window,1)
couple_input_shape =(window,2)
single_output_shape = (telescope,1)
couple_output_shape=(telescope,2)
batch_size = 64
epochs = 200

In [ ]:
models=[]
l_r=[0.001,0.00005,0.05,0.005,0.01]
for i in range(5):
    if (i <3):
        model = build_CONV_LSTM_model(single_input_shape, single_output_shape,learning_rate=l_r[i])
    else:
        model= build_CONV_LSTM_model(couple_input_shape, couple_output_shape,learning_rate=l_r[i])
        
    model.summary()
    tfk.utils.plot_model(model, expand_nested=True)
    models.append(model)

In [ ]:
# Train the models
i=0
for seq in train_sequences:
    history = models[i].fit(
        x = seq[0],
        y = seq[1],
        batch_size = batch_size,
        epochs = epochs,
        validation_split=.1,
        callbacks = [
            tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=4, factor=0.5,min_lr=0.0000001)]).history
    i=i+1
models[0].save('Sponginess')
models[1].save('Meme_Creativity')
models[2].save("soap_splipperiness")
models[3].save("crunch_hype")
models[4].save('wonder_lound')


In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
X_temp = single_datasets[0]
X_temp=X_temp[-300:]
#print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#print(X_temp.shape)
reg_future = np.array([])
#X_temp = future
for reg in range(0,864,telescope):
    pred_temp = models[0].predict(X_temp)
    #print(pred_temp)
    if(len(reg_future)==0):
        reg_future = pred_temp
    else:
        reg_future = np.concatenate((reg_future,pred_temp),axis=1)
    X_temp = tf.concat([X_temp[:, telescope:, :], pred_temp], axis=1)
out=reg_future[0,:864]


In [ ]:
#figs, axs = plt.subplots(2, 1, sharex=True, figsize=(17,5))
X_temp = single_datasets[0]
X_temp=X_temp[-300:]
print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#for i in range(2):
plt.figure(figsize=(17,2.5))
plt.plot(np.arange(len(X_temp[0,:])), X_temp[0,:])
plt.plot(np.arange(len(X_temp[0,]), len(X_temp[0,:])+864), out[:], color='orange')
#axs[i].fill_between(
   # np.arange(len(future[0,:,i]), len(future[0,:,i])+reg_telescope), 
   # reg_future[0,:,i]+maes[:,i], 
   # reg_future[0,:,i]-maes[:,i], 
    #color='orange', alpha=.3)
    #axs[i].set_title(names[0])
plt.title(names[0])
   # axs[i].set_ylim(0,1)
plt.show()

In [ ]:
X_temp = single_datasets[1]
X_temp=X_temp[-300:]
#print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#print(X_temp.shape)
reg_future = np.array([])
#X_temp = future
for reg in range(0,864,telescope):
    pred_temp = models[1].predict(X_temp)
    #print(pred_temp)
    if(len(reg_future)==0):
        reg_future = pred_temp
    else:
        reg_future = np.concatenate((reg_future,pred_temp),axis=1)
    X_temp = tf.concat([X_temp[:, telescope:, :], pred_temp], axis=1)
out=reg_future[0,:864]


In [ ]:
#figs, axs = plt.subplots(2, 1, sharex=True, figsize=(17,5))
X_temp = single_datasets[1]
X_temp=X_temp[-300:]
print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#for i in range(2):
plt.figure(figsize=(17,2.5))
plt.plot(np.arange(len(X_temp[0,:])), X_temp[0,:])
plt.plot(np.arange(len(X_temp[0,]), len(X_temp[0,:])+864), out[:], color='orange')
#axs[i].fill_between(
   # np.arange(len(future[0,:,i]), len(future[0,:,i])+reg_telescope), 
   # reg_future[0,:,i]+maes[:,i], 
   # reg_future[0,:,i]-maes[:,i], 
    #color='orange', alpha=.3)
    #axs[i].set_title(names[0])
plt.title(names[1])
   # axs[i].set_ylim(0,1)
plt.show()

In [ ]:
X_temp = single_datasets[2]
X_temp=X_temp[-300:]
#print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#print(X_temp.shape)
reg_future = np.array([])
#X_temp = future
for reg in range(0,864,telescope):
    pred_temp = models[2].predict(X_temp)
    #print(pred_temp)
    if(len(reg_future)==0):
        reg_future = pred_temp
    else:
        reg_future = np.concatenate((reg_future,pred_temp),axis=1)
    X_temp = tf.concat([X_temp[:, telescope:, :], pred_temp], axis=1)
out=reg_future[0,:864]

In [ ]:
#figs, axs = plt.subplots(2, 1, sharex=True, figsize=(17,5))
X_temp = single_datasets[2]
X_temp=X_temp[-300:]
print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#for i in range(2):
plt.figure(figsize=(17,2.5))
plt.plot(np.arange(len(X_temp[0,:])), X_temp[0,:])
plt.plot(np.arange(len(X_temp[0,]), len(X_temp[0,:])+864), out[:], color='orange')
#axs[i].fill_between(
   # np.arange(len(future[0,:,i]), len(future[0,:,i])+reg_telescope), 
   # reg_future[0,:,i]+maes[:,i], 
   # reg_future[0,:,i]-maes[:,i], 
    #color='orange', alpha=.3)
    #axs[i].set_title(names[0])
plt.title(names[2])
   # axs[i].set_ylim(0,1)
plt.show()

In [ ]:
X_temp = couple_datasets[0]
X_temp=X_temp[-300:]
#print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#print(X_temp.shape)
print(X_temp.shape)
print(models[1])
reg_future = np.array([])
#X_temp = future
for reg in range(0,864,telescope):
    pred_temp = models[3].predict(X_temp)
    #print(pred_temp)
    if(len(reg_future)==0):
        reg_future = pred_temp
    else:
        reg_future = np.concatenate((reg_future,pred_temp),axis=1)
    X_temp = tf.concat([X_temp[:, telescope:, :], pred_temp], axis=1)
out=reg_future[0,:864]
print(out)

In [ ]:
figs, axs = plt.subplots(2, 1, sharex=True, figsize=(17,5))
X_temp = couple_datasets[0]
X_temp=X_temp[-300:]
print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
for i in range(2):
#plt.figure(figsize=(17,2.5))
    axs[i].plot(np.arange(len(X_temp[0,:,i])), X_temp[0,:,i])
    axs[i].plot(np.arange(len(X_temp[0,:,i]), len(X_temp[0,:,i])+864), out[:,i], color='orange')
#axs[i].fill_between(
   # np.arange(len(future[0,:,i]), len(future[0,:,i])+reg_telescope), 
   # reg_future[0,:,i]+maes[:,i], 
   # reg_future[0,:,i]-maes[:,i], 
    #color='orange', alpha=.3)
    axs[i].set_title(names[3])
#plt.title(names[2])
   # axs[i].set_ylim(0,1)
plt.show()

In [ ]:
X_temp = couple_datasets[1]
X_temp=X_temp[-300:]
#print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
#print(X_temp.shape)
print(X_temp.shape)
print(models[1])
reg_future = np.array([])
#X_temp = future
for reg in range(0,864,telescope):
    pred_temp = models[4].predict(X_temp)
    #print(pred_temp)
    if(len(reg_future)==0):
        reg_future = pred_temp
    else:
        reg_future = np.concatenate((reg_future,pred_temp),axis=1)
    X_temp = tf.concat([X_temp[:, telescope:, :], pred_temp], axis=1)
out=reg_future[0,:864]
print(out)

In [ ]:
figs, axs = plt.subplots(2, 1, sharex=True, figsize=(17,5))
X_temp = couple_datasets[1]
X_temp=X_temp[-300:]
print(X_temp.shape)
#X_temp=np.reshape(X_temp,(window,-1))
#print(X_temp.shape)
X_temp=np.expand_dims(X_temp, axis=0)
for i in range(2):
#plt.figure(figsize=(17,2.5))
    axs[i].plot(np.arange(len(X_temp[0,:,i])), X_temp[0,:,i])
    axs[i].plot(np.arange(len(X_temp[0,:,i]), len(X_temp[0,:,i])+864), out[:,i], color='orange')
#axs[i].fill_between(
   # np.arange(len(future[0,:,i]), len(future[0,:,i])+reg_telescope), 
   # reg_future[0,:,i]+maes[:,i], 
   # reg_future[0,:,i]-maes[:,i], 
    #color='orange', alpha=.3)
    axs[i].set_title(names[4])
#plt.title(names[2])
   # axs[i].set_ylim(0,1)
plt.show()